**Livrable** : Un ou des notebooks (ou des fichiers .py) contenant les fonctions permettant le prétraitement et la feature extraction des données textes et images ainsi que les résultats de l’étude de faisabilité (graphiques, mesure de similarité) 

In [58]:
%%capture
!python -m spacy download en_core_web_sm
!python -m spacy download en_core_web_md
!python -m spacy download en_core_web_trf

In [59]:
import os
import pandas as pd
import numpy as np

# Lecture du jeu de données flipkart

In [60]:
# Read the content of the flipkart.csv file
flipkart = pd.read_csv('pj1_flipkart.csv', encoding='utf-8', encoding_errors='replace', parse_dates=['crawl_timestamp'])

In [61]:
# Print the information of the flipkart DataFrame
print(flipkart.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1050 entries, 0 to 1049
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype              
---  ------                   --------------  -----              
 0   uniq_id                  1050 non-null   object             
 1   crawl_timestamp          1050 non-null   datetime64[ns, UTC]
 2   product_url              1050 non-null   object             
 3   product_name             1050 non-null   object             
 4   product_category_tree    1050 non-null   object             
 5   pid                      1050 non-null   object             
 6   retail_price             1049 non-null   float64            
 7   discounted_price         1049 non-null   float64            
 8   image                    1050 non-null   object             
 9   is_FK_Advantage_product  1050 non-null   bool               
 10  description              1050 non-null   object             
 11  product_rating           1050 

# Commentaires sur le fichier flipkart après visualisation sur le data wrangler

| Champs                    | Description                                                                                  | Utile |
|---------------------------|----------------------------------------------------------------------------------------------|-------|
| uniq_id                   | Nom des fichiers images dans le répertoire image                                             | O     |
| crawl_timestamp           | Timestamp du crawler qui a récupéré les images                                               | N     |
| product_url               | Lien URL contenant le product_name & le PID                          | N     |
| product_name              | Nom du produit extrait de l'URL                                                             | O     |
| product_category_tree     | Fil d'ariane à extraire de la catégorie de produit categorical                               | O     |
| pid                       | Product ou Page ID                                                                                   | N     |
| retail_price              | Prix                                                                                         | O     |
| discounted_price          | Prix réduit                                                                                  | O     |
| image                     | La même chose que uniq_id mais avec le format à la fin                                       | O     |
| is_FK_Advantage_product   | Série booléenne de ?                                                                         | N     |
| description               | Description string du produit                                                                | O     |
| product_rating            | Note moyenne, attention les valeurs nulles sont remplacées par 'No rating available'         | O     |
| overall_rating            | Semble identique                                                                             | N     |
| brand                     | La marque, contient beaucoup de valeurs manquantes, peut être récupéré du product name       | N     |
| product_specifications    | Sous forme de K-V pair sur les key feature du produit                                        | O     |


# Vérification sur les champs

En plus du fichier flipkart, nous avons un répertoire contenant les images des produits. <br>
**Nous allons vérifier si les images dans le répertoire correspondent aux images dans le fichier flipkart** pour pouvoir ensuite rapprocher le dataset avec les images.

In [62]:
# Count the number of files in the current directory
print("Nombre de fichiers dans le répertoire Images: ", len(os.listdir("./Images")))

# Check if the number of lines in the flipkart.csv file is equal to the number of files in the Images directory
assert (
    len(os.listdir("./Images")) == len(flipkart)
), "Le nombre de lignes ne corresponds pas au nombre de fichiers dans le repertoire Images"

# Check if the uniq_id column has the same names than the files in the Images directory
files_without_jpg = [file.strip(".jpg") for file in os.listdir("./Images")]
assert (
    flipkart["uniq_id"].isin(files_without_jpg).all()
), "Les fichiers dans le repertoire Images ne correspondent pas aux fichiers dans le fichier flipkart.csv"
print("La colonne uniq_id contient les mêmes noms que les fichiers dans le répertoire Images")


Nombre de fichiers dans le répertoire Images:  1050
La colonne uniq_id contient les mêmes noms que les fichiers dans le répertoire Images


In [6]:
# Check if the image column is equal to the uniq_id column with the .jpg extension
assert flipkart["uniq_id"].equals(
    flipkart["image"].str.strip(".jpg")
), "les colonnes image sans l'extension .jpg et uniq_id ne correspondent pas"
print("La colonne image est égale à la colonne uniq_id sans l'extension .jpg")


La colonne image est égale à la colonne uniq_id sans l'extension .jpg


In [7]:
# Check if the image column contains different image formats
print("Les formats d'images sont:")
flipkart['image'].str.rsplit('.', expand=True)[1].value_counts()

Les formats d'images sont:


1
jpg    1050
Name: count, dtype: int64

In [8]:
# Check if the product_rating column is equal to the overall_rating column
assert flipkart["product_rating"].equals(
    flipkart["overall_rating"]
), "les colonnes product_rating et overall_rating ne correspondent pas"
print("La colonne product_rating est égale à la colonne overall_rating")


La colonne product_rating est égale à la colonne overall_rating


# Extraction des catégories de produits

**Nous allons extraire les catégories de produits pour pouvoir les utiliser dans la classification des produits.**<br>
Chaque catégorie est séparée par un '>>' et nous allons les séparer et les stocker dans une colonne 'category_1', 'category_2', 'category_3' etc...<br>
La colonne 'category_1' contient la catégorie principale, 'category_2' la sous-catégorie et 'category_3' la sous-sous-catégorie, etc...

In [9]:
# Extract the categories from the product_category_tree column into separate columns
categories = (
    flipkart["product_category_tree"].str.strip('["|"]')
    .str.split(">>", expand=True)
    .rename(columns={i: f'category_{i}' for i in range(0, 7)})
    )

# Concatenate the flipkart and categories dataframes and drop the product_category_tree column
flipkart_with_cat = (
    pd.concat([flipkart, categories], axis=1)
    .drop("product_category_tree", axis=1)
)

# Sélection des colonnes utiles

**Pour la suite de l'analyse, on va sélectionner les colonnes utiles pour la classification des produits.**<br>
On travaillera sur les colonnes 'product_name', 'description' et 'product_specifications' pour les données textes et sur les colonnes 'image' pour les données images.<br>
Concernant la colonne 'product_specifications', il est encore difficile de savoir si l'on va l'utiliser pour la classification des produits. Nous allons donc la conserver pour le moment.

In [10]:
fields = [
    "uniq_id",
    "product_name",
    "category_0",
    "category_1",
    "category_2",
    "category_3",
    "category_4",
    "category_5",
    "category_6",
    "retail_price",
    "discounted_price",
    "description",
    "product_rating",
    "product_specifications"
]

J'appellerai le nouveau dataframe `flipkart_filtered` qui contiendra les colonnes utiles pour la classification des produits.	

In [11]:
# Create a new DataFrame with the selected fields
flipkart_filtered = flipkart_with_cat[fields].copy()

# Remplacement des ratings nuls

In [12]:
# Replace the text "No rating available" by 0 in the product_rating column and convert it to float
flipkart_filtered = flipkart_filtered.assign(**{'product_rating': lambda x: x['product_rating'].replace('No rating available', np.nan).astype(float)})

# Identification des erreurs d'encodage dans les colonnes textes

J'ai identifié un problème d'encodage dans les colonnes textuelles avec le caractère �.<br>
Regardons les colonnes textuelles pour vérifier la présence de ce caractère.

In [13]:
# Define the problematic pattern
pattern = "�"

# Check if the flipkart_filtered DataFrame contains the character �
flipkart_filtered.select_dtypes(include='object').apply(lambda x: x.str.count(pattern).sum())

uniq_id                    0.0
product_name               3.0
category_0                 0.0
category_1                 0.0
category_2                 0.0
category_3                 0.0
category_4                 1.0
category_5                 0.0
category_6                 0.0
description               11.0
product_specifications     4.0
dtype: float64

In [14]:
# Create a new list
words = []
non_words = []

# Create a pattern to match words containing the character � with a pattern like "___" which doesn't exist in the text
word_pattern = r"\b\w*___\w*\b"
nonword_pattern = r"\b\W*___\W*\b"

# Add each word with word_pattern to the words list and each word with nonword_pattern to the non_words list
for column in flipkart_filtered.select_dtypes(include='object').replace('�', '___'):
    words.append((column, flipkart_filtered[column].dropna().str.replace('�', '___').str.findall(word_pattern).sum()))
    non_words.append((column, flipkart_filtered[column].dropna().str.replace('�', '___').str.findall(nonword_pattern).sum()))

# Print the words and non_words lists
print(words)
print(non_words)

[('uniq_id', []), ('product_name', ['Repouss______', '___']), ('category_0', []), ('category_1', []), ('category_2', []), ('category_3', []), ('category_4', ['___']), ('category_5', []), ('category_6', []), ('description', ['Repouss______', 'Repouss______', '___To', '___', '___To', '___', '___', 'd___cor', 'd___cor']), ('product_specifications', ['Repouss______', 'Repouss______'])]
[('uniq_id', []), ('product_name', [' ___ ']), ('category_0', []), ('category_1', []), ('category_2', []), ('category_3', []), ('category_4', [' ___ ']), ('category_5', []), ('category_6', []), ('description', ['.___\r\n', '.___', ' ___ ']), ('product_specifications', [])]


Après inspection des passages concernée, je peux remplacer les passages spécifiques avec "Repouss��" et "d�cor" par "Repousse" et "decor" respectivement.<br>
Pour le reste, je vais supprimer les occurences de ce caractère.

In [15]:
# Define the pattern to match
pattern_1 = "Repouss��"
pattern_2 = "d�cor"

# Replace the pattern_1 by "Repoussé" and the pattern_2 by "décor" in the product_specifications column
for column in flipkart_filtered.select_dtypes(include='object'):
    flipkart_filtered[column] = flipkart_filtered[column].str.replace(pattern_1, "Repoussé").str.replace(pattern_2, "décor").str.replace(pattern, '')

# Check if the flipkart_filtered DataFrame contains the character �
assert flipkart_filtered.select_dtypes(include='object').apply(lambda x: x.str.count(pattern).sum()).sum() == 0.0, "Il reste encore des caractères � dans le DataFrame"

# EDA sur les colonnes


In [16]:
flipkart_filtered

,uniq_id,product_name,category_0,category_1,category_2,category_3,category_4,category_5,category_6,retail_price,discounted_price,description,product_rating,product_specifications
0,55b85ea15a1536d46b7190ad6fff8ce7,Elegance Polyester Multicolor Abstract Eyelet ...,Home Furnishing,Curtains & Accessories,Curtains,Elegance Polyester Multicolor Abstract Eyelet...,None,None,None,1899.0,899.0,Key Features of Elegance Polyester Multicolor ...,NaN,"{""product_specification""=>[{""key""=>""Brand"", ""v..."
1,7b72c92c2f6c40268628ec5f14c6d590,Sathiyas Cotton Bath Towel,Baby Care,Baby Bath & Skin,Baby Bath Towels,Sathiyas Baby Bath Towels,"Sathiyas Cotton Bath Towel (3 Bath Towel, Red...",None,None,600.0,449.0,Specifications of Sathiyas Cotton Bath Towel (...,NaN,"{""product_specification""=>[{""key""=>""Machine Wa..."
2,64d5d4a258243731dc7bbb1eef49ad74,Eurospa Cotton Terry Face Towel Set,Baby Care,Baby Bath & Skin,Baby Bath Towels,Eurospa Baby Bath Towels,Eurospa Cotton Terry Face Towel Set (20 PIECE...,None,None,NaN,NaN,Key Features of Eurospa Cotton Terry Face Towe...,NaN,"{""product_specification""=>[{""key""=>""Material"",..."
3,d4684dcdc759dd9cdf41504698d737d8,SANTOSH ROYAL FASHION Cotton Printed King size...,Home Furnishing,Bed Linen,Bedsheets,SANTOSH ROYAL FASHION Bedsheets,SANTOSH ROYAL FASHION Cotton Printed King siz...,None,None,2699.0,1299.0,Key Features of SANTOSH ROYAL FASHION Cotton P...,NaN,"{""product_specification""=>[{""key""=>""Brand"", ""v..."
4,6325b6870c54cd47be6ebfbffa620ec7,Jaipur Print Cotton Floral King sized Double B...,Home Furnishing,Bed Linen,Bedsheets,Jaipur Print Bedsheets,Jaipur Print Cotton Floral King sized Double ...,None,None,2599.0,698.0,Key Features of Jaipur Print Cotton Floral Kin...,NaN,"{""product_specification""=>[{""key""=>""Machine Wa..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1045,958f54f4c46b53c8a0a9b8167d9140bc,Oren Empower Extra Large Self Adhesive Sticker,Baby Care,Baby & Kids Gifts,Stickers,Oren Empower Stickers,None,None,None,1399.0,999.0,Oren Empower Extra Large Self Adhesive Sticker...,NaN,"{""product_specification""=>[{""key""=>""Number of ..."
1046,fd6cbcc22efb6b761bd564c28928483c,Wallmantra Large Vinyl Sticker Sticker,Baby Care,Baby & Kids Gifts,Stickers,Wallmantra Stickers,None,None,None,4930.0,1896.0,Wallmantra Large Vinyl Sticker Sticker (Pack o...,NaN,"{""product_specification""=>[{""key""=>""Number of ..."
1047,5912e037d12774bb73a2048f35a00009,Uberlyfe Extra Large Pigmented Polyvinyl Films...,Baby Care,Baby & Kids Gifts,Stickers,Uberlyfe Stickers,None,None,None,4500.0,1449.0,Buy Uberlyfe Extra Large Pigmented Polyvinyl F...,NaN,"{""product_specification""=>[{""key""=>""Number of ..."
1048,c3edc504d1b4f0ba6224fa53a43a7ad6,Wallmantra Medium Vinyl Sticker Sticker,Baby Care,Baby & Kids Gifts,Stickers,Wallmantra Stickers,None,None,None,3465.0,1333.0,Buy Wallmantra Medium Vinyl Sticker Sticker fo...,NaN,"{""product_specification""=>[{""key""=>""Number of ..."


In [17]:
for i in range(1, 7):
    print("Number of categories in category_{}:".format(i))
    print(flipkart_filtered[f'category_{i}'].nunique())

Number of categories in category_1:
63
Number of categories in category_2:
246
Number of categories in category_3:
350
Number of categories in category_4:
297
Number of categories in category_5:
117
Number of categories in category_6:
57


In [18]:
# Concatenante the product_name and description columns
flipkart_filtered["text"] = flipkart_filtered["product_name"] + " " + flipkart_filtered["description"]

# CV - TF-IDF sur les colonnes textes

In [19]:
# Préparation des données textes pour encode avec CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
# Création d'une instance de TfIdfVectorizer
vectorizer = TfidfVectorizer()

In [20]:
# Encodage des données textes
X = vectorizer.fit_transform(flipkart_filtered["text"])

In [21]:
# Récupération du nom des features
feature_names = vectorizer.get_feature_names_out()

# Création d'un DataFrame avec les données encodées
text_encoded = pd.DataFrame(X.toarray(), columns=feature_names)

In [22]:
text_encoded

,00,000,001,0021,004,005,006,0083,01,011,...,zinc,zingalalaa,zip,zipexterior,zipper,zippered,zone,zoom,zora,zyxel
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1045,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1046,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1047,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1048,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
text_encoded.sum(axis=0).sort_values(ascending=False)

of             39.887814
only           37.715187
for            35.120723
at             32.578276
and            31.970817
                 ...    
safely          0.028043
trip            0.028043
portability     0.028043
348             0.028043
notepad         0.028043
Length: 6052, dtype: float64

In [24]:
import spacy


In [32]:
# Load the English tokenizer
nlp = spacy.blank("en")

In [33]:
# Print the attributes of the spacy pipeline
print(nlp.pipe_names)

[]


In [27]:
%pip install curated_transformers

Note: you may need to restart the kernel to use updated packages.


In [46]:
# Load the English spacy Roberta model
nlp = spacy.load("en_core_web_trf")

In [47]:
# Print the attributes of the spacy pipeline
print(nlp.pipe_names)

['transformer', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']


In [36]:
# Remove every attribute of the spacy pipeline except the lemmatizer
nlp.disable_pipes("transformer", "ner")


['transformer', 'ner']

In [37]:
nlp.pipe_names

['tagger', 'parser', 'attribute_ruler', 'lemmatizer']

In [ ]:
def spacy_preprocessor(doc):
    return " ".join([token.text for token in nlp(doc) if not token.is_stop and token.is_alpha])

In [52]:
def spacy_preprocessor(doc):
    return " ".join([token.lemma_.lower() for token in nlp(doc) if not token.is_stop and token.is_alpha])

In [53]:
tfidf = TfidfVectorizer(preprocessor=spacy_preprocessor, min_df=5, strip_accents='unicode')

In [ ]:
TfidfVectorizer?

Init signature:
TfidfVectorizer(
    *,
    input='content',
    encoding='utf-8',
    decode_error='strict',
    strip_accents=None,
    lowercase=True,
    preprocessor=None,
    tokenizer=None,
    analyzer='word',
    stop_words=None,
    token_pattern='(?u)\\b\\w\\w+\\b',
    ngram_range=(1, 1),
    max_df=1.0,
    min_df=1,
    max_features=None,
    vocabulary=None,
    binary=False,
    dtype=<class 'numpy.float64'>,
    norm='l2',
    use_idf=True,
    smooth_idf=True,
    sublinear_tf=False,
)
Docstring:     
Convert a collection of raw documents to a matrix of TF-IDF features.

Equivalent to :class:`CountVectorizer` followed by
:class:`TfidfTransformer`.

For an example of usage, see
:ref:`sphx_glr_auto_examples_text_plot_document_classification_20newsgroups.py`.

For an efficiency comparison of the different feature extractors, see
:ref:`sphx_glr_auto_examples_text_plot_hashing_vs_dict_vectorizer.py`.

For an example of document clustering and comparison with
:class:`~sklea

In [49]:
text = "he loves working with data. It works like a charm at his work"

In [50]:
doc = nlp(text)

In [51]:
for token in doc:
    print(token.text, token.lemma_, token.is_stop, token.is_alpha)

he he True True
loves love False True
working work False True
with with True True
data datum False True
. . False False
It it True True
works work False True
like like False True
a a True True
charm charm False True
at at True True
his his True True
work work False True


In [54]:
# Encodage des données textes
X = tfidf.fit_transform(flipkart_filtered["text"])

In [55]:
# Récupération du nom des features
feature_names = tfidf.get_feature_names_out()

# Création d'un DataFrame avec les données encodées
text_encoded = pd.DataFrame(X.toarray(), columns=feature_names)

In [56]:
text_encoded

,absorbent,abstract,ac,accessory,actual,adapter,add,addition,additional,adhesive,...,wireless,woman,wood,wooden,work,world,wrap,year,yellow,yes
0,0.0,0.212054,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.055310,0.0,0.000000,0.000000,0.000000
1,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.173857,0.068128
2,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.040182,0.0,0.0,0.0,0.000000,0.0,0.047634,0.000000,0.000000
3,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.042168
4,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.034755
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1045,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.090722,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000
1046,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.119800,...,0.0,0.000000,0.0,0.0,0.0,0.109059,0.0,0.000000,0.000000,0.000000
1047,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000
1048,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000


In [ ]:
text_encoded.sum(axis=0).sort_values(ascending=False)

watch          41.628355
analog         37.200037
products       31.595215
genuine        31.328765
shipping       31.328765
                 ...    
notepad         0.032455
expandable      0.032455
safely          0.032455
confidently     0.032455
distribute      0.032455
Length: 4727, dtype: float64

In [ ]:
text_encoded['aa'].sum()

0.1845161307808877

El